In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

import pandas as pd
import util.utilities as util
from collections import Counter

In [2]:
df_sms_extraction = util.load_dataset(my_path='../../dataset/Extraction_form_basic.xlsx')
df_sms_extraction = util.add_column_sp(df_sms_extraction)

# 3.3) Q2 (Measure of ATD)

## Papers grouped by Measure

dict_q2 = {}
for item in zip( list(df_sms_extraction['Paper Id']), list(df_sms_extraction['Q2(Measure)']) ):
    id = item[0]
    if (item[1] != ''):
        q2 = str(item[1])
        q2 = ' '.join(q2.split())
        dict_q2[id] = q2.lower()

In [3]:
## Papers grouped by kind of measurement

# 1) RQ2 - measurement of ATD - The measure of ATD is defined as the amount of identified ATD item
grouped_by_measurement = [(1, 'amount of software architecture rules violated in documents'), 
(2, 'amount of complexity measures of files'), 
(3, 'amount of architecture smells'), 
(4, 'amount of architectural root'),
(5, 'coupling of files and architecture smells'),
(7, 'amount of hotspots'),
(9, 'modularity metrics'),
(10, 'amount of software architecture rules violated in documents and source code'), 
(13, 'amount of architecture smells'),
(14, 'amount of hotspots'),
(15, 'amount of software architecture rules violated in source code'),
(16, 'amount of contagious debt'),
(21, 'amount of software architecture rules violated in documents'),
(22, 'index of crisis point'),
(27, 'modularity metrics'), 
(30, 'amount of architecture smells'),
(38, 'amount of architectural root and index of Decoupling Level'),
(48, 'amount of hotspots'),
(20, 'amount of architecture smells'),
(33, 'amount of software architecture rules violated in documents and source code'),
(28, 'modularity metrics'),
(31, 'dependency of components'),
(43, 'amount of complexity measures of files'), 
(51, 'amount of architecture smells'),
(52, 'amount of architecture smells'),
(63, 'other'),
(73, 'amount of software architecture rules violated in documents and source code'),
(35, 'index of crisis point'),
(24, 'amount of architectural root'),
(81, 'amount of software architecture rules violated in source code'),
(85, 'amount of software architecture rules violated in source code'),
(88, 'amount of software architecture rules violated in source code'),
(40, 'amount of software architecture rules violated in documents'),
(107, 'amount of architecture smells'),
 (108, 'other'),
 (109, 'amount of software architecture rules violated in documents and source code'),
 (110, 'modularity metrics'),
 (111, 'amount of software architecture rules violated in source code'),
 (112, 'amount of complexity measures of files'),
 (113, 'amount of complexity measures of files'),
 (114, 'amount of software architecture rules violated in source code'),
 (115, 'amount of architecture smells'),
 (116, 'amount of architecture smells')]

In [4]:
df_distribution_q2 = util.view_question_distribution(my_dict=dict_q2, my_list=['Measured', 'No', 'Not applicable'], my_question='q2')
df_distribution_q2['sp'] = None

df_distribution_q2 = util.view_question_distribution_update_sp(my_df_distribution=df_distribution_q2, my_df_data=df_sms_extraction, my_feature='Q2(Measure)')

util.create_latex_table(my_df=df_distribution_q2, my_path='../../latex', my_file_name='mytable_q2_distribution.tex')

df_distribution_q2

Arquivo ../../latex/mytable_q2_distribution.tex  gerado com sucesso!
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,q2,count,sp
0,Measured,44,"[SP1, SP2, SP3, SP4, SP5, SP6, SP7, SP8, SP9, SP10, SP11, SP12, SP14, SP15, SP16, SP17, SP18, SP19, SP21, SP22, SP24, SP25, SP27, SP28, SP29, SP30, SP31, SP36, SP37, SP40, SP41, SP42, SP44, SP46, SP48, SP49, SP50, SP51, SP52, SP53, SP54, SP55, SP56, S..."
1,No,13,"[SP13, SP20, SP23, SP26, SP32, SP33, SP34, SP35, SP38, SP39, SP43, SP45, SP47]"
2,Not applicable,0,[]


In [5]:
print('Measured : {} ({} studies)'.format( round(43/57 , 3), 43 ))
print('No measured : {} ({} studies)'.format( round(14/57 , 3), 14 ))

Measured : 0.754 (43 studies)
No measured : 0.246 (14 studies)


In [6]:
# 2) Add new column grouped_by_measurement
df_sms_extraction['grouped_by_measurement'] = None

# 3) Updating df_sms_extraction['grouped_by_measurement']
for each in grouped_by_measurement:
    j = 0
    for item in df_sms_extraction['Paper Id']:
        if each[0] == item: 
            df_sms_extraction['grouped_by_measurement'].iloc[j] = each[1]
        j = j + 1

# 4) Creating list_by_measurement using count 
list_by_measurement = []
for each in grouped_by_measurement: 
    list_by_measurement.append(each[1])

list_by_measurement = Counter(list_by_measurement).most_common()

# 5) Creating DataFrame df_my_measure
measure = []
count = []
for each in list_by_measurement: 
    measure.append(each[0])
    count.append(each[1])
my_measure = {'measure': measure, 'count':count}
my_measure

my_measure_id = list(range(1,len(list_by_measurement)+1))
my_measure_id

df_my_measure = pd.DataFrame(data=my_measure, index=my_measure_id)
df_my_measure.reset_index(drop=True, inplace=True)
df_my_measure['sp'] = None

# 6) Add new column with Selected Papers according grouped measurement
i = 0
for each in df_my_measure['measure']:
    list_sp_novo = []
    j = 0
    for item in df_sms_extraction['grouped_by_measurement']:
        if each == item: 
            list_sp_novo.append(df_sms_extraction['sp'].iloc[j])
        j = j + 1
    df_my_measure['sp'].iloc[i] = list_sp_novo
    i = i + 1

# 7) Save the latex table 
util.create_latex_table(my_df=df_my_measure, my_path='../../latex', my_file_name='mytable_q2_distribution_detailed.tex')

df_my_measure

Arquivo ../../latex/mytable_q2_distribution_detailed.tex  gerado com sucesso!


,measure,count,sp
0,amount of architecture smells,9,"[SP3, SP9, SP17, SP21, SP28, SP29, SP48, SP56, SP57]"
1,amount of software architecture rules violated in source code,6,"[SP11, SP40, SP41, SP42, SP52, SP55]"
2,amount of complexity measures of files,4,"[SP2, SP27, SP53, SP54]"
3,modularity metrics,4,"[SP7, SP16, SP24, SP51]"
4,amount of software architecture rules violated in documents and source code,4,"[SP8, SP22, SP31, SP50]"
5,amount of software architecture rules violated in documents,3,"[SP1, SP14, SP46]"
6,amount of hotspots,3,"[SP6, SP10, SP19]"
7,amount of architectural root,2,"[SP4, SP37]"
8,index of crisis point,2,"[SP15, SP36]"
9,other,2,"[SP30, SP49]"


In [7]:
print(df_my_measure['count'].sum())

43


In [8]:
df_temp = df_my_measure.query('count >= 3')[['measure', 'count']]
df_temp

,measure,count
0,amount of architecture smells,9
1,amount of software architecture rules violated in source code,6
2,amount of complexity measures of files,4
3,modularity metrics,4
4,amount of software architecture rules violated in documents and source code,4
5,amount of software architecture rules violated in documents,3
6,amount of hotspots,3


In [9]:
my_total = 57
for each in df_temp.values:
    my_measure = each[0]
    my_count = each[1]
    print('{} {}\% ({} studies)'.format(my_measure, round(my_count*100/my_total , 2), my_count ))

amount of architecture smells 15.79\% (9 studies)
amount of software architecture rules violated in source code 10.53\% (6 studies)
amount of complexity measures of files 7.02\% (4 studies)
modularity metrics 7.02\% (4 studies)
amount of software architecture rules violated in documents and source code 7.02\% (4 studies)
amount of software architecture rules violated in documents 5.26\% (3 studies)
amount of hotspots 5.26\% (3 studies)
